In [109]:
# import library
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering
%matplotlib inline 

# import data
dic = scipy.io.loadmat('USPS_fine_tuned_inceptionV3_by_MNIST_sorted.mat')
usps_features_noised = dic['USPS'].T
dic = scipy.io.loadmat('USPS_labels_sorted.mat')
usps_labels = dic['label']
dic = scipy.io.loadmat('USPS_feature_denoised_RBF.mat')
usps_features_denoised_RBF = dic['feats_denoised'].T
dic = scipy.io.loadmat('USPS_feature_denoised_cos.mat')
usps_features_denoised_cos = dic['feats_denoised'].T
dic = scipy.io.loadmat('cos_aff_mat.mat')
cos_aff_mat = dic['W']
dic = scipy.io.loadmat('cos_aff_mat_denoised.mat')
cos_aff_mat_denoised = dic['W']
dic = scipy.io.loadmat('RBF_aff_mat.mat')
RBF_aff_mat = dic['W']
dic = scipy.io.loadmat('RBF_aff_mat_denoised.mat')
RBF_aff_mat_denoised = dic['W']

# setting up models
spectral_clustering_RBF_raw = SpectralClustering(n_clusters = 10, affinity = 'rbf')
spectral_clustering_cos_raw = SpectralClustering(n_clusters = 10, affinity = 'cosine')
spectral_clustering_RBF_precompute = SpectralClustering(n_clusters=10, affinity = 'precomputed')
spectral_clustering_cos_precompute = SpectralClustering(n_clusters=10, affinity = 'precomputed')



In [17]:
# After Denoising, 2 ways to check for Accuracy
# 1. Spectral Clustering
# 1.1 Vanilla Cosine Affinity
# 1.1.1 run SpectralClustering directly
spectral_clustering_cos_raw.fit(usps_features_denoised_cos)
cos_spectral_raw_labels = spectral_clustering_cos_raw.fit_predict(usps_features_denoised_cos)
# 1.1.2 run based on precomputed affinity matrx
spectral_clustering_cos_precompute.fit(cos_aff_mat_denoised)
cos_spectral_labels = spectral_clustering_cos_raw.fit_predict(usps_features_denoised_cos)

#TODO which error metric should I use? 

In [9]:
# 1.2 RBF Affinity


# 1.3 Tensor Voting (No Reversed_Sparse_ball function)

In [69]:
from keras.models import Model, load_model, Sequential
from keras.layers import Dense

In [25]:
# 2. Passing through the last layer of the network 
base_model = load_model('model.h5')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [26]:
for i, layer in enumerate(base_model.layers):
    print (i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [76]:
weights = base_model.get_layer('dense_2').get_weights()
# since cannot conveniently calculate the result, 
# we calculate it manually

# first calculate x*W + b
# verify the dimension 
print (weights[0].shape)
print (weights[1].shape)
print (usps_features_denoised_cos.shape)

(1024, 10)
(10,)
(2000, 1024)


In [112]:
# Calculation
logits_noi = np.matmul(usps_features_noised, weights[0]) + weights[1]
logits_cos = np.matmul(usps_features_denoised_cos, weights[0]) + weights[1]
logits_RBF = np.matmul(usps_features_denoised_RBF, weights[0]) + weights[1]
print (logits_cos.shape)
print (logits_RBF.shape)

(2000, 10)
(2000, 10)


In [116]:
def sftmx(logits):
    exp = np.exp(logits)
    sum_e = np.sum(exp, axis = 1, keepdims = True)
    return exp/sum_e

softmax_cos = sftmx(logits_cos)
softmax_RBF = sftmx(logits_RBF)
softmax_noi = sftmx(logits_noi)

In [126]:
# Calculating Accuracy 
prediction_cos = np.argmax(softmax_cos, axis = 1)
prediction_RBF = np.argmax(softmax_RBF, axis = 1)
prediction_noi = np.argmax(softmax_noi, axis = 1)
ground_truth = usps_labels[0]

In [131]:
def accuracy(prediction, ground_truth):
    return (np.sum(prediction == ground_truth)/prediction.shape[0])
accuracy_cos = accuracy(prediction_cos, ground_truth)
accuracy_RBF = accuracy(prediction_RBF, ground_truth)
accuracy_noi = accuracy(prediction_noi, ground_truth)

In [132]:
print ("accuracy for cosine affinity: {0}".format(accuracy_cos))
print ("accuracy for RBF affinity: {0}".format(accuracy_RBF))
print ("accuracy for no denoising: {0}".format(accuracy_noi))

accuracy for cosine affinity: 0.0305
accuracy for RBF affinity: 0.027
accuracy for no denoising: 0.037


In [50]:
layer.get_config()

{'activation': 'softmax',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'dense_2',
 'trainable': True,
 'units': 10,
 'use_bias': True}